In [ ]:
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult, ValidationType
from src.projects.fagradalsfjall.common.model_eval_plot_curves import generate_all_rmse_plots
from src.projects.fagradalsfjall.common.model_eval_plot_simulations import plot_simulations
from src.projects.fagradalsfjall.common.model_repo import ModelRepo
from src.projects.fagradalsfjall.common.paths import get_blog_post_subfolder
from src.projects.fagradalsfjall.common.project_settings import TS_PRIMARY_METRIC_DISPLAY_NAME
from src.tools.datetime import format_timedelta

# 3. Visualize results

In [ ]:
# -------------------------------------------------------------------------
#  Output path settings
# -------------------------------------------------------------------------
path_results_mlp_mse = get_blog_post_subfolder(6, "results_1_mlp_mse")

In [ ]:
# -------------------------------------------------------------------------
#  Load data
# -------------------------------------------------------------------------

# --- load models -------------------------------------
prev_model_eval_results = ModelRepo.load_models(["naive-mean", "naive-constant", "naive-exp-decay", "linear-ar"])

# load mlp-mse results
mlp_mse_model_eval_results = ModelRepo.load_models(["mlp-rmse"])
mlp_mse_n_1_model_eval_results = ModelRepo.load_models(["mlp-rmse-n-1"])
mlp_mse_n_192_model_eval_results = ModelRepo.load_models(["mlp-rmse-n-192"])

# rename
mlp_mse_model_eval_results = {"mlp-mse": mlp_mse_model_eval_results["mlp-rmse"]}
mlp_mse_n_1_model_eval_results = {"mlp-mse-n-1": mlp_mse_n_1_model_eval_results["mlp-rmse-n-1"]}
mlp_mse_n_192_model_eval_results = {"mlp-mse-n-192": mlp_mse_n_192_model_eval_results["mlp-rmse-n-192"]}

# --- post-processing ---------------------------------
model_mlp_mse = mlp_mse_model_eval_results["mlp-mse"].ts_model
model_mlp_mse_n_1 = mlp_mse_n_1_model_eval_results["mlp-mse-n-1"].ts_model
model_mlp_mse_n_192 = mlp_mse_n_192_model_eval_results["mlp-mse-n-192"].ts_model

In [ ]:
# -------------------------------------------------------------------------
#  Overall grid search 1D results
# -------------------------------------------------------------------------
param_names = ["n_epochs", "n_hidden_layers", "dropout", "n"]

for param_name in param_names:

    fig, ax = (
        model_mlp_mse.cv.results.plot_1d(param_names=param_name)
        .set_fig_title(f"Cross-validation - 'mlp-mse' - best results for each value of '{param_name}'")
        .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
        .set_x_label(param_name)
        .create(w=12, h=8)
    )
    fig.savefig(path_results_mlp_mse / f"cv_1d_best_{param_name}.png", dpi=600)

In [ ]:
# -------------------------------------------------------------------------
#  RMSE curves for n=1 & 192
# -------------------------------------------------------------------------

# merge all evaluation results in single dict for visualization
model_eval_results = prev_model_eval_results | mlp_mse_n_1_model_eval_results | mlp_mse_n_192_model_eval_results

# create RMSE plots
generate_all_rmse_plots(model_eval_results, path_results_mlp_mse, highlight_models=["mlp-mse-n-1", "mlp-mse-n-192"])

In [ ]:
# -------------------------------------------------------------------------
#  Simulations for n=1 & n=192
# -------------------------------------------------------------------------

# simulations
plot_simulations(mlp_mse_n_1_model_eval_results | mlp_mse_n_192_model_eval_results, path_results_mlp_mse)

In [ ]:
df_metrics = ModelEvalResult.all_metric_values_as_df(model_eval_results, ValidationType.TEST)

df_metrics["max_accurate_lead_time_samples"] = df_metrics["max_accurate_lead_time"].copy()
df_metrics["max_accurate_lead_time"] = df_metrics["max_accurate_lead_time"].apply(
    lambda x: format_timedelta(x * 15 * 60)
)

display(df_metrics)

In [ ]:
# -------------------------------------------------------------------------
#  Final parameters
# -------------------------------------------------------------------------

for n, model in [(1, model_mlp_mse_n_1), (192, model_mlp_mse_n_192)]:

    print()
    print("=================================")
    print(f" n={n}")
    print("=================================")
    print(f"metric: {model_mlp_mse.cv.results.filter(param_filter=dict(n=n)).best_result.val_metrics.overall:.3f}")
    print()

    for param_name, param_value in model.get_params().items():
        print(f"{param_name} : ".rjust(20) + str(param_value))